# SLU15 - Hyperparameter tunning: Examples notebook

## 1. Load and prepare the data

In [1]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

cancer_data = load_breast_cancer()
X = pd.DataFrame(cancer_data["data"], columns=cancer_data["feature_names"])
y = cancer_data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=0)
estimator = DecisionTreeClassifier()

## 2. Grid search

In [2]:
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth': range(1, 10),
              'max_features': range(1, X.shape[1])}

grid_search = GridSearchCV(estimator, parameters, cv=5, scoring="roc_auc")
grid_search.fit(X_train, y_train)

y_pred = grid_search.predict(X_test)

## 3. Random search

In [3]:
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

parameters_dist = {"max_depth": randint(1, 100),
                   "max_features": randint(1, X.shape[1]),
                   "class_weight": ["balanced", None]}

random_search = RandomizedSearchCV(estimator, parameters_dist, cv=5, n_iter=250, 
                                   random_state=0)

random_search.fit(X_train, y_train)
y_pred = random_search.predict(X_test)

## 4. Model selection by crossvalidation

In [4]:
# example for the grid search
from sklearn.model_selection import cross_validate

grid_results = cross_validate(grid_search.best_estimator_, X_test, y_test, scoring="roc_auc", 
                              return_train_score=True, cv=3)
pd.DataFrame(grid_results).mean()

fit_time       0.003709
score_time     0.005283
test_score     0.921356
train_score    0.988846
dtype: float64

In [5]:
# example for the random search
random_results = cross_validate(random_search.best_estimator_,
                                X_test, y_test, scoring="roc_auc",
                                return_train_score=True, cv=3)
pd.DataFrame(random_results).mean()

fit_time       0.004128
score_time     0.003631
test_score     0.902462
train_score    1.000000
dtype: float64